# 3 Zenith-afhankelijkheid van een station.

Dit notebook sluit aan op het notebook 'HiSPARC_API'. Er wordt zowel informatie van het station als van de metingen verwerkt.

Een HiSPARC-station meet deeltjes in een deeltjeslawine die wordt veroorzaakt door een primair kosmisch deeltje. De deeltjes in de lawine bewegen globaal met de lichtsnelheid. De interactie van het primaire deeltje vindt plaats op een hoogte van tientallen kilometers. De doorsnede van de deeltjeslawine is slechts enkele hectometers. We kunnen we er dus van uitgaan dat deeltjes globaal in een vlak loodrecht op de snelheid van het primaire deeltje bewegen.

In [1]:
import numpy as np
import sapphire
from sapphire import Station
detectors = Station(501).detectors()
for detector in detectors:
    print detector

{u'alpha': -158.07, u'beta': 45.0, u'radius': 6.09, u'height': 0.0}
{u'alpha': 73.49, u'beta': 45.0, u'radius': 5.0, u'height': 0.0}
{u'alpha': 95.63, u'beta': 315.0, u'radius': 14.51, u'height': 0.0}
{u'alpha': 138.13, u'beta': 315.0, u'radius': 11.06, u'height': 0.0}


De eigenschappen van een station zijn gedefinieerd met:

- 'mpv' : Most Probable Value. De spanning waarbij de grafiek van het aantal pulsen als functie van de spanning horizontaal is. Zie bijvoorbeeld: [http://data.hisparc.nl/show/stations/3401/2015/11/23/](http://data.hisparc.nl/show/stations/3401/2015/11/23/).
- 'alpha' : De hoek tussen de as gedefinieerd door de GPS-antenne en het midden van de scintillatorplaat en het Noorden. 
- 'beta' : De hoek tussen de lange zijde van de scintillatorplaat en het Noorden.
- 'radius' : De afstand van GPS- antenne tot het midden van de detector.

Een werkblad voor het maken van een stations-plattegrond is te vinden op:

[http://docs.hisparc.nl/infopakket/pdf/station_map.pdf](http://docs.hisparc.nl/infopakket/pdf/station_map.pdf)

De afstand tussen twee detectoren is te bepalen met de cosinusregel. Dit is in een functie te beschrijven:

In [2]:
def afstand(detector_1, detector_2):
    '''Bepaal de afstand tussen twee detectoren'''
    c = detector_2['radius']
    b = detector_1['radius']
    alpha = np.radians(detector_2['alpha'] - detector_1['alpha'])
    return (b ** 2 + c ** 2 - 2 * b * c * np.cos(alpha)) ** .5

De afstand tussen detector 1 (telt als 0) en 4 (telt als 3) is nu te berekenen met:

In [3]:
print afstand(detectors[0], detectors[3])

9.99680415762


In het notebook python_data_retrieval is uitgelegd hoe data op te halen is:

In [4]:
from sapphire import quick_download
data = quick_download(508)

100%|########################################################################################################|Time: 0:00:06


In [5]:
events = data.root.s508.events.read()
events = events[0:100] # bewaar alleen eerste 100 events

In [6]:
tijdkolommen = ['t1', 't2', 't3', 't4'] 
for kolom in tijdkolommen:
    print data.root.s508.events[0][kolom]

-999.0
-999.0
12.5
1232.5


In [7]:
from itertools import combinations

def zenithoeken(event, detectors):
    ''' 
    De zenithoek is de hoek tussen het golf-front en de horizon, 
    of ook tussen de as naar het zenith en de as van de deeltjeslawine.
    
    parameters
    event: een enkel event uit de opgehaalde data
    detectors: de detectorinformatie uit de API
    
    returns
    een array met de hoek tussen de as door twee detectors en het deeltjes front.
    '''
    c = 0.2997 # in m/ns
    
    tijden = (event['t1'], event['t2'], event['t3'], event['t4'])
    detector_tijd_paren = zip(detectors, tijden)
    
    zenith = []
    for paar1, paar2 in combinations(detector_tijd_paren, 2):
            detector1, tijd1 = paar1
            detector2, tijd2 = paar2
            
            schuine = afstand(detector1, detector2)
            overstaande = c * (tijd1 - tijd2) / 100
            
            angle = np.degrees(np.arcsin(overstaande / schuine))
            zenith.append(angle)
    return zenith

In [8]:
event = events[0]
print zenithoeken(event, detectors)

[0.0, -10.127260720918814, -41.989495631400928, -17.54380272472331, -41.985099228887279, -21.884367024641541]


In [9]:
for event in events[0:10]:
    print zenithoeken(event, detectors)

[0.0, -10.127260720918814, -41.989495631400928, -17.54380272472331, -41.985099228887279, -21.884367024641541]
[17.69637885523958, 10.152555987129873, 0.0, 0.0, -17.696029048917396, -18.047024662012479]
[-0.3864606551253239, 10.152555987129873, 17.697587712249359, 17.992049672271168, 18.102136040321998, 0.0]
[17.651311956911581, 10.127260720918814, -0.34354275106450388, 0.0, -18.0118094865404, -18.369510707622069]
[0.0, -22.878871924219862, -17.652517531292158, -41.797067329758818, -17.650963100473881, 21.978720398147068]
[-20.147774303487374, -10.127260720918814, 0.0, 2.3484130411150321, 20.14737206792622, 18.001003730557112]
[48.128491944025797, 25.582396133001112, 26.196390724091909, 0.0, -17.650963100473881, -18.001003730557112]
[-17.69637885523958, -17.64810045061699, 0.0, -12.564918034855134, 17.696029048917396, 32.196675917809536]
[-0.17175924604736087, 0.12450096797435845, -0.17177060360193872, 0.38418105956126514, 0.0, -0.39386597301568416]
[20.02889906949337, 21.98987630401512

Hierboven zijn de gereconstrueerde hoeken voor de eerste 10 events voor alle combinaties van 2 detectoren in een set van 4 detectoren berekend. De hoeken zijn gedefinieerd als de hoek tussen de as door de detector en het hart van de deeltjeslawine.

Een beschrijving van de zenith-hoek is te vinden op: [http://docs.hisparc.nl/infopakket/pdf/richting_reconstructie.pdf](http://docs.hisparc.nl/infopakket/pdf/richting_reconstructie.pdf).